In [7]:
from ipywidgets import *
import numpy as np
import pandas as pd
import ipywidgets as widgets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from ipywidgets import interact

shape = pd.read_csv('Shape.csv')

x_data_1 = np.array(shape['t_vortex_test'])
x_data_2 = np.array(shape['mPAP'])
shape_params = shape.iloc[:, 1:15]
y_data = np.array((shape_params)).T

selected_columns = shape.iloc[:, 1:15].columns

# Convert the column names to a list of strings
params = selected_columns.tolist()


In [8]:
from scipy.stats import linregress

r_squared_t =[]
r_squared_p =[]

for i in range(len(params)):

    slope, intercept, r_value_1, p_value, std_err = linregress(y_data[i], x_data_1)
    r2_1 = r_value_1**2
    slope, intercept, r_value_2, p_value, std_err = linregress(y_data[i], x_data_2)
    r2_2 = r_value_2**2
    
    r_squared_t.append(r2_1)
    r_squared_p.append(r2_2)
    
data = {'Parameters': params,
        'R_squared_T': r_squared_t,
       'R_squared_P': r_squared_p}

df = pd.DataFrame(data)


In [3]:
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import ipywidgets as widgets
from ipywidgets import interact

def interactive_parametric_plot(x_data, y_data_sets, methods):
    """
    Creates an interactive plot with different tested parameters

    Args:
        x_data: t_vortex for each test
        y_data_sets: A list of lists or arrays, each representing a tested parameter.
    """

    def update_plot(**kwargs):
        """
        Updates the plot with the selected y-data sets and regression lines.

        Args:
          kwargs: A dictionary of checkbox names (e.g., 'checkbox_1') and their corresponding boolean values.
        """
        # Create a Plotly figure
        fig = go.Figure()

        for i, (checkbox_name, selected) in enumerate(kwargs.items()):
            if selected:
                label = methods[i]
                
                # Select corresponding x and y data
                x = x_data[i]
                y = y_data_sets[i]

                # Plot the data points
                fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name=label))
                
                
                # Perform linear regression
                reg = LinearRegression(fit_intercept=True).fit(x.reshape(-1, 1), y.reshape(-1, 1))
                y_reg = reg.predict(x.reshape(-1, 1))  # Get the predicted y-values for the regression line
                r_squared = r2_score(y, y_reg)

                # Add regression line
                fig.add_trace(go.Scatter(x=x, y=y_reg.flatten(), mode='lines', 
                                         name=f'{label} Regression, R2 = {r_squared:.2f}', line=dict(dash='dash')))

        # Update layout
        fig.update_layout(
            title='Parametric Study Results',
            xaxis_title='Normalized Vortex Duration',
            yaxis_title='Normalized Tested Parameter',
            template='plotly_white',
            showlegend=True
        )

        fig.show()

    # Create checkboxes for each data set
    checkboxes = {}
    for i, y_data_set in enumerate(y_data_sets):
        checkbox = widgets.Checkbox(value=True, description=str(methods[i]))
        checkboxes[f'checkbox_{i+1}'] = checkbox

    # Create the interactive plot
    interact(update_plot, **checkboxes)



n_tests = 9
results = pd.read_csv('Parametric_Results.csv')
col = results.columns

x_data = []
y_data = []
for t in range(n_tests):
    sel_col = col[t*2:t*2+2]
    test = np.array(results[sel_col].dropna())
    x_data.append(test[:, 0])
    y_data.append(test[:, 1])


shape_t = np.array(shape['t_vortex_test'])
norm_shape_t = (shape_t-shape_t.min())/(shape_t.max()-shape_t.min())
x_data.append(norm_shape_t)

shape_score = np.array(shape['Shape score'])
norm_shape_score = (shape_score-shape_score.min())/(shape_score.max()-shape_score.min())
y_data.append(norm_shape_score)

methods = ['Compliance', 'Flow split', 'Total Afterload', 'Inlet BC metric, Shape 1', 'Inlet BC metric, Shape 2', 'Inlet BC metric, Shape 3', 'Inlet BC metric, Shape 4', 'Inlet BC metric, Shape 5', 'Inlet Profile', 'Shape Score']


interactive_parametric_plot(x_data, y_data, methods)

interactive(children=(Checkbox(value=True, description='Compliance'), Checkbox(value=True, description='Flow s…

In [9]:
import plotly.graph_objects as go
from scipy.stats import linregress
import pandas as pd
import numpy as np
import ipywidgets as widgets
import anywidget

def interactive_selection_plot(df, y_arrays):
    def update_plot(*args):
        # Create the Figure
        fig = go.FigureWidget()
        filtered_df = pd.DataFrame()

        # Check which checkboxes are selected
        for i, checkbox in enumerate(checkbox_widgets):
            if checkbox.value:
                filtered_df = pd.concat([filtered_df, pd.DataFrame(df.loc[i]).T], ignore_index=True)
        
        # Create traces based on selected y-arrays
        traces = []
        selected_y_array = y_array_dropdown.value
        for y_array in y_arrays:
            if selected_y_array == y_array:
                traces.append(go.Bar(x=filtered_df['Parameters'],
                                     y=np.array(filtered_df[y_array]),
                                     name=y_array))
        
        # Add traces to the figure
        fig.add_traces(traces)
        
        # Customize Layout
        fig.update_layout(
            title='R2 between Geometrical Features and mPAP or vortex duration:<br>L = Length, Curv = Curvature, D = Diameter',
            xaxis_title='Parameter',
            yaxis_title='R2',
            width=900,  # Plot area width
            height=600,  # Plot area height
        )
        
        # Clear the output and display the figure
        with plot_output:
            plot_output.clear_output(wait=True)
            fig.show()
    
    # Create checkboxes for each data set
    global checkbox_widgets
    checkbox_widgets = [widgets.Checkbox(value=True, description=str(parameter)) for parameter in df['Parameters']]
    
    # Arrange checkboxes in a grid
    grid_columns = 3  # Number of columns in the grid
    checkbox_grid = widgets.GridBox(
        checkbox_widgets,
        layout=widgets.Layout(
            grid_template_columns=f"repeat({grid_columns}, 1fr)",  # Create a grid with `grid_columns` columns
            gap="10px"  # Space between grid items
        )
    )
    
    # Create dropdown for selecting Y-array
    global y_array_dropdown
    y_array_dropdown = widgets.Dropdown(
        options=y_arrays,
        description='Y-array:',
        value=y_arrays[0]  # Default selection
    )
    
    # Create an output widget with a larger size for the plot window
    global plot_output
    plot_output = widgets.Output(layout=widgets.Layout(width='1200px', height='800px'))  # Adjust size here
    
    # Attach event handlers
    for checkbox in checkbox_widgets:
        checkbox.observe(lambda change: update_plot(), names='value')
    y_array_dropdown.observe(lambda change: update_plot(), names='value')
    
    # Combine controls and plot into a layout
    controls_layout = widgets.VBox([checkbox_grid, y_array_dropdown])
    display(widgets.VBox([controls_layout, plot_output]))
    
    # Initialize the interactive plot
    update_plot()

y_arrays = ['R_squared_T', 'R_squared_P']

interactive_selection_plot(df, y_arrays)